In [1]:
import sys
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import torch
import numpy as np
from sklearn.cluster import HDBSCAN
import random
from tqdm import tqdm
# from lib.dataloader import CustomTestDataset
from boilerplate.dataloader import Custom3DTestDataset
import tifffile as tiff
import os
from boilerplate import boilerplate
from scipy.spatial.distance import cdist
from torch.nn.functional import unfold
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

dist_metric = ['cosine']

num_clusters = 4
patch_size = (32,64,64)
centre_size = 5
n_channel = 32
hierarchy_level = 3

In [3]:
data_dir = "/group/jug/Sheida/pancreatic beta cells/download/"

One_test_image = ['high_c4']

# Load test image
test_img_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_source.tif")
test_images = tiff.imread(test_img_path)

# Print loaded test images paths
print("Test image loaded from path:")
print(test_img_path)

# Load test ground truth images
test_gt_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_gt.tif")
test_ground_truth_image = tiff.imread(test_gt_path)

Test image loaded from path:
/group/jug/Sheida/pancreatic beta cells/download/high_c4/high_c4_source.tif


In [4]:
def count_label_pixels(ground_truth):
    # Get unique labels and their counts
    unique_labels, counts = np.unique(ground_truth, return_counts=True)
    
    # Create a dictionary to store label-wise pixel counts
    label_pixel_counts = dict(zip(unique_labels, counts))
    
    return label_pixel_counts

In [5]:
label_pixel_counts = count_label_pixels(test_ground_truth_image)
print(f"Label-wise pixel counts:{label_pixel_counts}")

Label-wise pixel counts:{-1: 409001638, 0: 107710351, 1: 25910981, 2: 36936035, 3: 23104175}


In [6]:
def select_random_pixels_within_range(ground_truth, labels, pixels_per_class, x_range, y_range, z_range):
    selected_pixels = {}

    for label in labels:
        # Get the coordinates where ground truth equals the label
        coords = np.argwhere(ground_truth == label)
        
        # Filter coordinates based on the specified range
        x_in_range = (coords[:, 2] >= x_range[0]) & (coords[:, 2] <= x_range[1])
        y_in_range = (coords[:, 1] >= y_range[0]) & (coords[:, 1] <= y_range[1])
        z_in_range = (coords[:, 0] >= z_range[0]) & (coords[:, 0] <= z_range[1])
        
        # Keep only the coordinates that satisfy all range conditions
        filtered_coords = coords[z_in_range & y_in_range & x_in_range]
        
        # Check if there are enough pixels for the current label after filtering
        if len(filtered_coords) < pixels_per_class:
            raise ValueError(f"Not enough pixels for label {label} within the specified range. Found {len(filtered_coords)}, required {pixels_per_class}")
        
        # Randomly select pixels_per_class coordinates from filtered_coords
        chosen_indices = np.random.choice(len(filtered_coords), pixels_per_class, replace=False)
        chosen_coords = filtered_coords[chosen_indices]
        
        # Store selected coordinates for the label
        selected_pixels[label] = chosen_coords

    return selected_pixels


In [7]:
labels_of_interest = [0, 1, 2, 3]
z , y, x = test_images.shape
x_range = (64, x - 64)
y_range = (64, y - 64)
z_range = (64, z - 64)


In [8]:
selected_pixel_coords = select_random_pixels_within_range(test_ground_truth_image, labels_of_interest, pixels_per_class=256, x_range=x_range, y_range=y_range, z_range=z_range)

In [9]:
for label, coords in selected_pixel_coords.items():
    print(f"Label {label}: {coords}")

Label 0: [[ 804  308  471]
 [ 511  268  251]
 [ 684  171  532]
 [ 743  369  273]
 [ 772  418  688]
 [ 560  412  427]
 [ 651  377  500]
 [ 565  195  563]
 [ 502  343  855]
 [ 250  332  898]
 [ 596  109  618]
 [ 768  243  687]
 [ 582  410  196]
 [ 746  414  945]
 [ 788  170  614]
 [ 498  102  764]
 [ 716  444  713]
 [ 803  207  282]
 [ 402  194  384]
 [ 765  359  780]
 [ 806  424  912]
 [ 852  370  814]
 [ 645  316  153]
 [ 636  455  852]
 [ 798  316  688]
 [ 175  398  794]
 [ 736  174  670]
 [ 686  400  199]
 [ 337  204  879]
 [ 220  262  513]
 [ 226  343  441]
 [ 331  151  940]
 [ 724  152  577]
 [ 645  450  698]
 [ 491  172  479]
 [ 704  443  322]
 [ 237  139 1012]
 [ 639  450  736]
 [ 809  307  568]
 [ 343  289  889]
 [ 821  263  256]
 [ 638  399  850]
 [ 890  328  690]
 [ 810  213  370]
 [ 847  459  758]
 [ 631  410  222]
 [ 679  415  424]
 [ 535  175  268]
 [ 376  251  843]
 [ 212  384  613]
 [ 696  284  129]
 [ 654  258  599]
 [ 638  464  550]
 [ 219  373  900]
 [ 522  127  353]
 

In [10]:
import pickle

def save_as_pickle(selected_pixel_coords, filename):
    with open(filename, 'wb') as f:
        pickle.dump(selected_pixel_coords, f)

# Usage
save_as_pickle(selected_pixel_coords, 'selected_pixel_coords.pkl')


In [11]:
def extract_patches(image, coords, patch_size=(64,64,64)):
    half_size_z = patch_size[0] // 2
    half_size_y = patch_size[1] // 2
    half_size_x = patch_size[2] // 2
    patches = []

    # Get the shape of the image
    z_max, y_max, x_max = image.shape

    for coord in coords:
        z, y, x = coord

        # Calculate the boundaries of the patch
        x_start = max(0, x - half_size_x)
        x_end = min(x_max, x + half_size_x)
        y_start = max(0, y - half_size_y)
        y_end = min(y_max, y + half_size_y)
        z_start = max(0, z - half_size_z)
        z_end = min(z_max, z + half_size_z)
        
        # Extract the patch from the image
        patch = image[z_start:z_end, y_start:y_end, x_start:x_end]
        
        # Check if the patch is the required size, otherwise pad with zeros
        if patch.shape != patch_size:
            print("something is wrong")

        
        # Store the patch
        patches.append(patch)

    return np.array(patches)

In [12]:
all_patches_tensors = []

for label, coords in selected_pixel_coords.items():
    patches = extract_patches(test_images, coords, patch_size=patch_size)
    patches_tensor = torch.tensor(patches, dtype=torch.float32).unsqueeze(1)
    print(f"Patches for label {label}: {patches_tensor.shape}")  # The shape should be (256, 8, 64, 64)
    all_patches_tensors.append(patches_tensor)

combined_tensor = torch.stack(all_patches_tensors, dim=0).reshape(64, 16, 1, 32, 64, 64)
print(combined_tensor.shape)

Patches for label 0: torch.Size([256, 1, 32, 64, 64])
Patches for label 1: torch.Size([256, 1, 32, 64, 64])
Patches for label 2: torch.Size([256, 1, 32, 64, 64])
Patches for label 3: torch.Size([256, 1, 32, 64, 64])
torch.Size([64, 16, 1, 32, 64, 64])


In [13]:
model_dir = "/group/jug/Sheida/HVAE/3D/"
model_versions = [4,5,6,7,8]
batch_size = 16
for model_v in model_versions:
    model = torch.load(model_dir+'v0'+str(model_v)+"/model/3D_HVAE_best_vae.net")
    data_mean = model.data_mean
    data_std = model.data_std
    model.mode_pred = True
    model.eval()
    device = model.device
    print(f"Processing 1000 pixels with model version {model_v}")
    index = 0 
    all_mus = np.zeros((1024, 43008), dtype=np.float16)
    with torch.no_grad():
        for batch in tqdm(combined_tensor):
            batch = batch.to(device)
            batch = (batch - data_mean) / data_std
            output = model(batch)
            mu_test = torch.cat([torch.mean(output["mu"][i],dim=2).reshape(batch_size, -1) for i in range(hierarchy_level)], dim=1)
            mu_test = np.array(mu_test.cpu().numpy())
            all_mus[index:index+batch_size] = mu_test
            index += batch_size

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(all_mus)
    cluster_labels = cluster_labels.reshape(4, 256)
    tiff.imwrite(f"{model_dir}v0{model_v}/1000pixels.tif", cluster_labels)

Processing 1000 pixels with model version 4


  0%|          | 0/64 [00:00<?, ?it/s]/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905969118/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(
/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradi

Processing 1000 pixels with model version 5


  0%|          | 0/64 [00:00<?, ?it/s]/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 64/64 [02:56<00:00,  2.76s/it]


Processing 1000 pixels with model version 6


  0%|          | 0/64 [00:00<?, ?it/s]/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 64/64 [02:56<00:00,  2.76s/it]


Processing 1000 pixels with model version 7


  0%|          | 0/64 [00:00<?, ?it/s]/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 64/64 [02:56<00:00,  2.75s/it]


Processing 1000 pixels with model version 8


  0%|          | 0/64 [00:00<?, ?it/s]/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sheida.rahnamai/miniconda3/envs/emseg/lib/python3.12/site-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 64/64 [02:56<00:00,  2.75s/it]


In [14]:
v4 = tiff.imread(f"{model_dir}v04/1000pixels.tif")
print(v4.shape)

(4, 256)


In [15]:
for i in range(4):
    print(count_label_pixels(v4[i]))

{1: 105, 2: 151}
{0: 59, 1: 196, 2: 1}
{1: 31, 2: 225}
{1: 215, 2: 40, 3: 1}


In [16]:
v5 = tiff.imread(f"{model_dir}v05/1000pixels.tif")
print(v4.shape)
for i in range(4):
    print(count_label_pixels(v5[i]))

(4, 256)
{1: 162, 2: 65, 3: 29}
{0: 218, 1: 31, 3: 7}
{1: 152, 2: 67, 3: 37}
{1: 233, 2: 12, 3: 11}


In [17]:
v6 = tiff.imread(f"{model_dir}v06/1000pixels.tif")
print(v6.shape)
for i in range(4):
    print(count_label_pixels(v6[i]))

(4, 256)
{0: 198, 1: 20, 2: 6, 3: 32}
{0: 250, 1: 1, 2: 1, 3: 4}
{0: 201, 1: 20, 2: 5, 3: 30}
{0: 204, 1: 16, 2: 11, 3: 25}


In [24]:
v7 = tiff.imread(f"{model_dir}v07/1000pixels.tif")
print(v7.shape)
for i in range(4):
    print(count_label_pixels(v7[i]))

(4, 256)
{0: 2, 1: 8, 2: 17, 3: 229}
{0: 149, 1: 85, 2: 22}
{0: 2, 2: 18, 3: 236}
{0: 4, 1: 12, 2: 14, 3: 226}


In [26]:
v8 = tiff.imread(f"{model_dir}v07/1000pixels.tif")
print(v8.shape)
for i in range(4):
    print(count_label_pixels(v8[i]))

(4, 256)
{0: 2, 1: 8, 2: 17, 3: 229}
{0: 149, 1: 85, 2: 22}
{0: 2, 2: 18, 3: 236}
{0: 4, 1: 12, 2: 14, 3: 226}


In [19]:
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment

def compute_dice_coefficient(ground_truth, predicted):
    """
    Compute Dice coefficient between ground truth and predicted segmentation after resolving label permutations.
    
    Args:
    - ground_truth (numpy array): Ground truth labels (values from 0 to 3).
    - predicted (numpy array): Predicted labels (after clustering, might be permuted).
    
    Returns:
    - dice_coefficients (list): Dice coefficients for each class (label).
    """
    # Ensure both arrays are 1D
    ground_truth = ground_truth.flatten()
    predicted = predicted.flatten()
    
    # Create a confusion matrix
    # Rows: Ground truth labels (0 to 3)
    # Columns: Predicted cluster labels (0 to 3, possibly permuted)
    conf_matrix = confusion_matrix(ground_truth, predicted, labels=[0, 1, 2, 3])
    
    # Use the Hungarian algorithm to find the optimal label assignment (best permutation)
    row_ind, col_ind = linear_sum_assignment(-conf_matrix)  # Negative because it's maximization problem
    
    # Re-map predicted labels to match ground truth
    remapped_predicted = np.zeros_like(predicted)
    for gt_label, pred_label in zip(row_ind, col_ind):
        remapped_predicted[predicted == pred_label] = gt_label
    
    # Compute Dice coefficient for each class
    dice_coefficients = []
    for label in np.unique(ground_truth):
        gt_binary = (ground_truth == label)
        pred_binary = (remapped_predicted == label)
        
        intersection = np.sum(gt_binary & pred_binary)
        dice = (2. * intersection) / (np.sum(gt_binary) + np.sum(pred_binary))
        dice_coefficients.append(dice)
    
    return dice_coefficients

In [20]:
gt = np.zeros((4, 256), dtype=np.uint8)
gt[0] = 0
gt[1] = 1
gt[2] = 2
gt[3] = 3

dice_scores = compute_dice_coefficient(gt, v4)
print(dice_scores)  # Dice coefficients for each class

[0.0, 0.3746031746031746, 0.6686478454680534, 0.5354919053549191]


In [21]:
dice_scores = compute_dice_coefficient(gt, v5)
print(dice_scores)  # Dice coefficients for each class

[0.325, 0.919831223628692, 0.21764705882352942, 0.5587529976019184]


In [22]:
dice_scores = compute_dice_coefficient(gt, v6)
print(dice_scores)  # Dice coefficients for each class

[0.1844380403458213, 0.4508566275924256, 0.12779552715654952, 0.07885304659498207]


In [25]:
dice_scores = compute_dice_coefficient(gt, v7)
print(dice_scores)  # Dice coefficients for each class

[0.10397553516819572, 0.7215496368038741, 0.498416050686378, 0.0664819944598338]


In [27]:
dice_scores = compute_dice_coefficient(gt, v8)
print(dice_scores)  # Dice coefficients for each class

[0.10397553516819572, 0.7215496368038741, 0.498416050686378, 0.0664819944598338]
